# Microbe-Set Enrichment Analysis

Microbe-Set Enrichment Analysis[MSEA](https://www.nature.com/articles/s41598-020-78511-y#Sec8) is a host-microbe enrichment analysis package that checks for enriched microbes with known microbe-human gene interactions reported in literature. This needs a reference microbe set compendium (the one used here is provided by the authors) and a list of microbes of interest basically the one that comes up as significant in the differential abundance analysis.

# install msea package
#pip install msea

# Step 1: Read reference

Read in a GMT file with reference human-microbe.Here, the file provided by msea is taken into consideration which has a total of 1286 microbe-sets (gene: microbes) from published literature.

In [32]:
from pprint import pprint
import msea
from msea import SetLibrary
import pandas

# 0. load a reference microbe-set library from a GMT file
gmt_filepath = \
    'https://bitbucket.org/wangz10/msea/raw/aee6dd184e9bde152b4d7c2f3c7245efc1b80d23/msea/data/human_genes_associated_microbes/set_library.gmt'

# msea package provides a `read_gmt` function to parse a GMT file into a
# dictionary of sets
d_gmt = msea.read_gmt(gmt_filepath)
print('Number of microbe-sets:', len(d_gmt))
# Look at a couple of reference sets in the library
pprint(list(d_gmt.items())[:3])

Number of microbe-sets: 1286
[('A2M', {'Pseudomonas', 'Sodalis', 'Salmonella', 'Borrelia', 'Azomonas'}),
 ('AAAS',
  {'Colwellia',
   'Deinococcus',
   'Idiomarina',
   'Neisseria',
   'Pseudidiomarina',
   'Pseudoalteromonas'}),
 ('AACS',
  {'Acetobacter',
   'Acinetobacter',
   'Azomonas',
   'Corynebacterium',
   'Enterobacter',
   'Klebsiella',
   'Mycobacterium',
   'Mycoplasma',
   'Pseudomonas',
   'Sodalis',
   'Staphylococcus',
   'Streptomyces',
   'Tetragenococcus'})]


# Step 2: msea.enrich
perform MSEA for a input microbe-set against the library of reference sets

In [4]:
#example
microbe_set_input = set(['Colwellia',
                         'Deinococcus',
                         'Idiomarina',
                         'Neisseria',
                         'Pseudidiomarina',
                         'Pseudoalteromonas'])
# this can be done using the `msea.enrich` function
msea_result = msea.enrich(microbe_set_input, d_gmt=d_gmt, universe=1000)
# check the top enriched reference microbe-sets
print(msea_result.head())

       oddsratio        pvalue        qvalue  \
term                                           
AAAS  166.666667  5.436249e-10  6.991016e-07   
NAGS   47.619048  1.478752e-07  9.508376e-05   
AGRP  111.111111  9.710341e-07  4.162500e-04   
FICD   95.238095  1.512723e-06  4.863405e-04   
SACS   27.777778  1.283717e-05  2.122535e-03   

                                                 shared  n_shared  
term                                                               
AAAS  [Deinococcus, Pseudidiomarina, Idiomarina, Col...         6  
NAGS  [Deinococcus, Pseudidiomarina, Idiomarina, Col...         6  
AGRP  [Pseudoalteromonas, Pseudidiomarina, Colwellia...         4  
FICD  [Idiomarina, Colwellia, Pseudidiomarina, Neiss...         4  
SACS  [Pseudidiomarina, Idiomarina, Colwellia, Pseud...         5  


# Step 3: msea with adjustment
Perform MSEA with adjustment of expected ranks for reference sets sometimes certain reference microbe-sets in a library are more likely to be enriched by chance. We can adjust this by empirically estimating the null distributions of the ranks of the reference sets, then uses z-score to quantify if observed ranks are significantly different from the expected ranks.

In [25]:
# To do that, it is easier through the `SetLibrary` class.
set_library = SetLibrary.load(gmt_filepath)
# The `SetLibrary.get_empirical_ranks` method helps compute the expected
# ranks and store the means and stds of the ranks from the null
# distribution
set_library.get_empirical_ranks(n=20)
print(set_library.rank_means.shape, set_library.rank_stds.shape)
# After that, we can perform MSEA with this adjustment
msea_result_adj = set_library.enrich(microbe_set_input, adjust=True, universe=1000)
print(msea_result_adj.head())

  0%|          | 0/20 [00:00<?, ?it/s]

Calculating empirical ranks for each set...
Number of unique microbes: 566


100%|██████████| 20/20 [00:13<00:00,  1.46it/s]


(1286,) (1286,)
      oddsratio    pvalue    qvalue    zscore  combined_score  \
term                                                            
ARSI  31.250000  0.000424  0.014727 -2.549074       19.797220   
RBFA  50.000000  0.000131  0.007639 -1.801593       16.111115   
CFI   35.714286  0.000302  0.012126 -1.714504       13.897729   
CCL2   8.547009  0.004440  0.098440 -2.557406       13.853860   
AGA   25.000000  0.000752  0.021204 -1.567378       11.273449   

                                                 shared  n_shared  
term                                                               
ARSI           [Idiomarina, Colwellia, Pseudidiomarina]         3  
RBFA           [Idiomarina, Colwellia, Pseudidiomarina]         3  
CFI            [Idiomarina, Colwellia, Pseudidiomarina]         3  
CCL2  [Idiomarina, Colwellia, Pseudidiomarina, Neiss...         4  
AGA            [Idiomarina, Colwellia, Pseudidiomarina]         3  


In [24]:
#import math
math.log10(0.000030)*(-1.707623)

7.723371771651845

# Create msea_pml()

In [27]:
def msea_pml(microbe_set_input, gmt_filepath):
    set_library = SetLibrary.load(gmt_filepath)
    set_library.get_empirical_ranks(n=20)
    print(set_library.rank_means.shape, set_library.rank_stds.shape)
    msea_result_adj = set_library.enrich(microbe_set_input, adjust=True, universe=1000)
    print(msea_result_adj.head())
    return msea_result_adj

# Cancer.vs.Ctrl
## Up Microbes

In [41]:
microbe_set_input = set(['Fusobacterium',
                         'Dialister',
                         'Peptostreptococcus',
                         'Prevotella',
                         'Shewanella',
                         'Campylobacter',
                         'Veillonella',
                         'Neisseria',
                         'Porphyromonas',
                         'Solobacterium',
                         'Glaesserella',
                         'Haemophilus',
                         'Microlunatus',
                         'Bergeyella',
                         'Lachnoanaerobaculum',
                         'Bibersteinia',
                         'Aggregatibacter',
                         'Obesumbacterium',
                         'Lancefieldella',
                         'Centipeda',
                         'Snodgrassella',
                         'Riemerella',
                         'Schaalia',
                         'Plesiomonas',
                         'Gemella',    
                         'Simonsiella',
                         'Leptotrichia'])
can_ctrl_up = msea_pml(microbe_set_input=microbe_set_input, gmt_filepath=gmt_filepath)
can_ctrl_up.to_csv('/restricted/projectnb/montilab-p/projects/oralcancer/pml/results/mmkhan/msea_cancer_ctrl_up.csv')

  0%|          | 0/20 [00:00<?, ?it/s]

Calculating empirical ranks for each set...
Number of unique microbes: 566


100%|██████████| 20/20 [00:13<00:00,  1.53it/s]
/usr4/bs849/mmkhan/.local/lib/python3.7/site-packages/msea/set_library.py:73: RuntimeWarning: 10 of the 27 microbes in the input_set do(es) not exist in the reference set library: Solobacterium;Bergeyella;Centipeda;Lancefieldella;Schaalia;Glaesserella;Simonsiella;Snodgrassella;Obesumbacterium;Lachnoanaerobaculum
  list(unmapped_items))), RuntimeWarning)


(1286,) (1286,)
       oddsratio    pvalue    qvalue    zscore  combined_score  \
term                                                             
IFI16  29.629630  0.000075  0.013524 -8.543537       81.176134   
IL15    5.446623  0.004915  0.072107 -7.980909       42.423027   
IL2     5.108557  0.013535  0.121663 -8.033760       34.565202   
CD86    5.050505  0.000408  0.020967 -4.267790       33.311064   
IL6     4.306632  0.000649  0.023920 -3.601049       26.430838   

                                                  shared  n_shared  
term                                                                
IFI16  [Campylobacter, Porphyromonas, Aggregatibacter...         4  
IL15   [Campylobacter, Shewanella, Aggregatibacter, H...         5  
IL2    [Porphyromonas, Prevotella, Aggregatibacter, N...         4  
CD86   [Veillonella, Campylobacter, Aggregatibacter, ...         9  
IL6    [Veillonella, Shewanella, Campylobacter, Aggre...        10  


## Dn Microbes

In [40]:
microbe_set_input = set(['Gardnerella',
                         'Limnohabitans',
                         'Kocuria',
                         'Ralstonia',
                         'Thalassospira',
                         'Agrobacterium', 
                         'Streptomyces',
                         'Delftia', 
                         'Bradyrhizobium',
                         'Streptomyces',
                         'Malassezia',
                         'Pseudoglutamicibacter',
                         'Rhodoluna',
                         'Tepidimonas'
                        ])
                            
can_ctrl_dn = msea_pml(microbe_set_input=microbe_set_input, gmt_filepath=gmt_filepath)
can_ctrl_dn.to_csv('/restricted/projectnb/montilab-p/projects/oralcancer/pml/results/mmkhan/msea_cancer_ctrl_dn.csv')

  0%|          | 0/20 [00:00<?, ?it/s]

Calculating empirical ranks for each set...
Number of unique microbes: 566


100%|██████████| 20/20 [00:15<00:00,  1.28it/s]
/usr4/bs849/mmkhan/.local/lib/python3.7/site-packages/msea/set_library.py:73: RuntimeWarning: 4 of the 13 microbes in the input_set do(es) not exist in the reference set library: Rhodoluna;Tepidimonas;Pseudoglutamicibacter;Malassezia
  list(unmapped_items))), RuntimeWarning)


(1286,) (1286,)
         oddsratio    pvalue    qvalue    zscore  combined_score  \
term                                                               
DST      30.769231  0.004065  0.746886 -2.517558       13.859721   
COL18A1  15.384615  0.012130  0.858285 -1.986376        8.764094   
CHIA      5.128205  0.032140  0.858285 -2.282949        7.848001   
COPA      5.917160  0.009655  0.858285 -1.627569        7.552413   
CCNB1    10.256410  0.023740  0.858285 -1.936983        7.245425   

                                                    shared  n_shared  
term                                                                  
DST                          [Streptomyces, Agrobacterium]         2  
COL18A1                      [Streptomyces, Agrobacterium]         2  
CHIA              [Ralstonia, Streptomyces, Agrobacterium]         3  
COPA     [Ralstonia, Streptomyces, Bradyrhizobium, Agro...         4  
CCNB1                        [Streptomyces, Agrobacterium]         2  


# PML.vs.Ctrl
## Up Microbes
Combining both HkNR+Dys microbes

In [39]:
microbe_set_input = set(['Shewanella',
                         'Aggregatibacter',
                         'Haemophilus',
                         'Mannheimia', 
                         'Veillonella',
                         'Dialister',
                         'Fusobacterium',
                         'Neisseria',
                         'Riemerella',
                         'Leptotrichia',
                         'Campylobacter', 
                         'Peptostreptococcus',
                         'Porphyromonas',
                         'Streptococcus',
                         'Plesiomonas',
                         'Bibersteinia', 
                         'Prevotella',
                         'Gemella',
                         'Microlunatus',
                         'Novosphingopyxis',
                         'Marivita',
                         'Oceanivirga',
                         'Simonsiella',
                         'Bergeyella',
                         'Snodgrassella',
                         'Solobacterium',
                         'Lachnoanaerobaculum',
                         'Mannheimia',
                         'Schaalia',
                         'Canicola' 
                         ])
pml_ctrl_up = msea_pml(microbe_set_input=microbe_set_input, gmt_filepath=gmt_filepath)
pml_ctrl_up.to_csv('/restricted/projectnb/montilab-p/projects/oralcancer/pml/results/mmkhan/msea_pml_ctrl_up.csv')

  0%|          | 0/20 [00:00<?, ?it/s]

Calculating empirical ranks for each set...
Number of unique microbes: 566


100%|██████████| 20/20 [00:14<00:00,  1.37it/s]
/usr4/bs849/mmkhan/.local/lib/python3.7/site-packages/msea/set_library.py:73: RuntimeWarning: 10 of the 29 microbes in the input_set do(es) not exist in the reference set library: Solobacterium;Oceanivirga;Schaalia;Canicola;Simonsiella;Marivita;Snodgrassella;Bergeyella;Novosphingopyxis;Lachnoanaerobaculum
  list(unmapped_items))), RuntimeWarning)


(1286,) (1286,)
       oddsratio    pvalue    qvalue    zscore  combined_score  \
term                                                             
IL1B    5.260082  0.000305  0.008844 -7.637203       61.820945   
CD5     9.852217  0.000015  0.002645 -1.920120       21.384082   
HLA-B  10.775862  0.000370  0.009698 -2.670712       21.107465   
DPP4    9.074410  0.000710  0.013837 -2.891157       20.961066   
CD40    5.945303  0.000065  0.006427 -2.081086       20.064909   

                                                  shared  n_shared  
term                                                                
IL1B   [Campylobacter, Shewanella, Aggregatibacter, F...         9  
CD5    [Campylobacter, Aggregatibacter, Fusobacterium...         8  
HLA-B  [Aggregatibacter, Streptococcus, Leptotrichia,...         5  
DPP4   [Aggregatibacter, Fusobacterium, Streptococcus...         5  
CD40   [Riemerella, Veillonella, Campylobacter, Aggre...        10  


## Dn Microbes

In [38]:
# not found: Curvibacter;Centipeda;Lancefieldella;Catonella;Rhodoluna;Lacrimispora;Cronobacter
microbe_set_input = set(['Agrobacterium',
                         'Selenomonas',
                         'Rhodoluna'
                        ])
pml_ctrl_dn = msea_pml(microbe_set_input=microbe_set_input, gmt_filepath=gmt_filepath)
pml_ctrl_dn.to_csv('/restricted/projectnb/montilab-p/projects/oralcancer/pml/results/mmkhan/msea_pml_ctrl_dn.csv')

  0%|          | 0/20 [00:00<?, ?it/s]

Calculating empirical ranks for each set...
Number of unique microbes: 566


100%|██████████| 20/20 [00:14<00:00,  1.34it/s]
/usr4/bs849/mmkhan/.local/lib/python3.7/site-packages/msea/set_library.py:73: RuntimeWarning: 1 of the 3 microbes in the input_set do(es) not exist in the reference set library: Rhodoluna
  list(unmapped_items))), RuntimeWarning)


(1286,) (1286,)
        oddsratio    pvalue  qvalue    zscore  combined_score  \
term                                                            
ARR3    55.555556  0.027476     1.0 -2.756625        9.908497   
BAX     20.833333  0.065112     1.0 -2.319484        6.336014   
ARG2    23.809524  0.057732     1.0 -1.916147        5.464729   
ALAD    16.666667  0.079655     1.0 -1.374613        3.477834   
BCL2L1  14.492754  0.090378     1.0 -1.310547        3.150238   

                 shared  n_shared  
term                               
ARR3    [Agrobacterium]         1  
BAX     [Agrobacterium]         1  
ARG2    [Agrobacterium]         1  
ALAD    [Agrobacterium]         1  
BCL2L1  [Agrobacterium]         1  
